In [23]:
!pip install mediapipe

In [25]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle

# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/ISL_Recognition/vit_model.h5')

load_path = "/content/drive/MyDrive/ISL_Recognition/label_encoder.pkl"
with open(load_path, 'rb') as f:
    label_encoder = pickle.load(f)

# Initialize MediaPipe Hands
import mediapipe as mp
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.7)

def adjust_gamma(image, gamma=1.5):
    """Adjust gamma of an image."""
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def extract_hand_landmarks(image):
    """
    Extracts 21 hand landmarks (x, y, z) from an image using MediaPipe Hands.

    Returns:
        - (63,) NumPy array if hand is detected
        - Zero-filled (63,) array if no hand is detected
    """
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image_rgb = adjust_gamma(image_rgb, gamma=1.5)  # Apply gamma correction
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0]
        landmarks_array = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark]).flatten()  # (63,)
    else:
        landmarks_array = np.zeros((21, 3)).flatten()  # (63,)

    return landmarks_array

def preprocess_image(image_path):
    """
    Preprocess an image for classification:
    - Load the image.
    - Extract hand landmarks.
    - Reshape and normalize the landmarks.
    """\

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Error: Unable to load image at {image_path}")

    # Extract hand landmarks
    landmarks = extract_hand_landmarks(image)

    # Reshape landmarks to match model input shape (21, 3)
    landmarks = landmarks.reshape(21, 3)

    # Normalize landmarks (if required)
    # landmarks = (landmarks - np.mean(landmarks, axis=0)) / np.std(landmarks, axis=0)

    # Add batch dimension
    landmarks = np.expand_dims(landmarks, axis=0)

    return landmarks

def classify_image(image_path):
    """
    Classify an image using the loaded model.
    """

    # Preprocess the image
    input_data = preprocess_image(image_path)

    # Make prediction
    predictions = model.predict(input_data)

    # Get the predicted class
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]

    # Get prediction probabilities
    prediction_probabilities = predictions[0]

    print(f"Predicted Class Index: {predicted_class_index}")
    print(f"Predicted Class: {predicted_class}")
    print(f"Prediction Probabilities: {prediction_probabilities}\n")

    return predicted_class, prediction_probabilities

# Example usage
image_path = "/content/drive/MyDrive/Miniproject (1)/Indian/Q/100.jpg"
predicted_class, probabilities = classify_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 951ms/step
Predicted Class Index: 25
Predicted Class: Q
Prediction Probabilities: [1.7845479e-05 3.2689441e-05 7.2164206e-05 7.9702033e-05 3.2041033e-05
 1.6825830e-05 4.7097761e-05 9.8852703e-05 1.1676957e-04 1.4563779e-04
 1.4685567e-01 1.9533221e-01 1.4764309e-04 1.2877966e-04 2.8924584e-05
 1.5920405e-04 2.6707316e-04 1.4552276e-04 2.2185413e-05 1.2089150e-04
 3.7551326e-05 5.0510116e-05 2.1681824e-05 1.3327636e-01 4.4199402e-04
 3.8297516e-01 6.9663693e-05 6.3012175e-02 7.2697883e-05 5.8515139e-05
 7.5597987e-02 8.5203073e-06 1.2399864e-04 1.8993372e-04 1.9544167e-04]

